## Fill in fields in PDF search results


In [59]:
# library containign read and write functions to csv file
import lib.handle_csv as csvh

# managing files and file paths
from pathlib import Path

# library for handling url searchs
import lib.handle_urls as urlh

# add a progress bar
from tqdm import tqdm_notebook
    
# library for accessing system functions
import os

# import custom functions (common to various notebooks)
import processing_functions as pr_fns

# date format
from datetime import date

# Connecting to the db
import lib.handle_db as dbh

# get the publications list from the app database
ukchapp_db = "./db_files/production.sqlite3"

In [2]:
# get names and links for references in data mentions
# data_reference, do_keys = csvh.get_csv_data('./data_search_pdf/new_references202111.csv')
working_file = './data_search_pdf/pdf_mentionsapp_db202204_checked.csv'
data_reference, do_keys = csvh.get_csv_data(working_file)
print(do_keys)
db_conn = dbh.DataBaseAdapter(ukchapp_db)

for dr in tqdm_notebook(data_reference):
    publication_title = db_conn.get_title(data_reference[dr]['doi'])
    # db keys:
    # 'dataset_complete', IGNORE
    # 'dataset_description', try to add, if not use publication title
    # get data from doi if available

    if data_reference[dr]['dataset_description'] == "":
        data_reference[dr]['dataset_description'] = data_reference[dr]['desc']
        if data_reference[dr]['desc'] == "":
            data_reference[dr]['doi'] = data_reference[dr]['doi'].lower().strip()
            print (data_reference[dr]['link'], "is missing description" )
            print("Description: Supplementary information for ", publication_title)
            data_reference[dr]['dataset_description'] = "Supplementary data for " + publication_title[0]
    # 'dataset_doi', if available
    # 'do_id', need new field?
    # 'dataset_enddate',IGNORE
    #' dataset_location', URL (could be id)
    if data_reference[dr]['dataset_location'] == "":
        print("missing location for ",data_reference[dr]['link'])
        data_reference[dr]['dataset_location'] = data_reference[dr]['link']
    # 'dataset_name', try to add, if not use DO type and pub doi/id
    if data_reference[dr]['dataset_name'] == "":
        print("missing name for ",data_reference[dr]['link'])
        data_reference[dr]['dataset_name'] = data_reference[dr]['type'] +" ("+ data_reference[dr]['type'] +") for article" 
    # 'dataset_startdate', try to add, use pub data if not available
    if data_reference[dr]['dataset_startdate'] == "":
        print("missing start data for ",data_reference[dr]['link'])
        #try to get date from DOI
        if data_reference[dr]['dataset_doi'] != "":
            print("look up for doi data")
            ref_link = "https://doi.org/" + data_reference[dr]['dataset_doi']
            data_object = urlh.getObjectMetadata(ref_link)
            if 'metadata' in data_object.keys():
                print(data_object['metadata'])
                data_reference[dr]['dataset_description'] = data_object['metadata']['abstract']
                if 'published' in data_object['metadata'].keys():
                    print(data_object['metadata']['published'])
        else:
            print("get date from publication")
            db_data = db_conn.get_row("Articles", data_reference[dr]['id'])
            art_poy = db_conn.get_value("articles", "pub_ol_year", "id", data_reference[dr]['id'])[0]
            art_pom = db_conn.get_value("articles", "pub_ol_month", "id", data_reference[dr]['id'])[0]
            art_pod = db_conn.get_value("articles", "pub_ol_day", "id", data_reference[dr]['id'])[0]
            art_ppy = db_conn.get_value("articles", "pub_print_year", "id", data_reference[dr]['id'])[0]
            art_ppm = db_conn.get_value("articles", "pub_print_month", "id", data_reference[dr]['id'])[0]
            art_ppd = db_conn.get_value("articles", "pub_print_day", "id", data_reference[dr]['id'])[0]
            if art_poy != '' and art_pom != '' and art_pod != '' and art_poy != None and art_pom != None and art_pod != None:
                print ("use online date: ", art_poy, art_pom, art_pod, art_ppy, art_ppm, art_ppd)
                data_reference[dr]['dataset_startdate'] = date(int(art_poy), int(art_pom), int(art_pod))
            elif art_poy != '' and art_pom != '' and art_poy != None and art_pom != None:
                print ("use online date: ", art_poy, art_pom, art_pod, art_ppy, art_ppm, art_ppd)
                data_reference[dr]['dataset_startdate'] = date(int(art_poy), int(art_pom), 1)
            elif art_ppy != '' and art_ppm != '' and art_ppd != '' and art_ppy != None and art_ppm != None and art_ppd != None:
                print ("use print date: ",art_ppy, art_ppm, art_ppd)
                data_reference[dr]['dataset_startdate'] = date(art_ppy, art_ppm, art_ppd)
            elif art_ppy != '' and art_ppm != '' and art_ppy != None and art_ppm != None:
                print ("use print date: ",art_ppy, art_ppm, 1)
                data_reference[dr]['dataset_startdate'] = date(art_ppy, art_ppm, 1)
            elif art_poy != '' and art_poy != None:
                print ("use online date: ", art_poy, 1, 1)
                data_reference[dr]['dataset_startdate'] = date(art_poy, 1, 1)
    
# write to csv file
if len(data_reference) > 0:
    csvh.write_csv_data(data_reference, working_file)        

['id', 'doi', 'type', 'desc', 'data_url', 'action', 'link', 'issue', 'name', 'file', 'add', 'num', 'dataset_complete', 'dataset_description', 'dataset_doi', 'do_id', 'dataset_enddate', 'dataset_location', 'dataset_name', 'dataset_startdate', 'ds_type', 'repository', 'ID', 'in_db']


C:\Users\scman1\AppData\Local\Temp\ipykernel_17256\3025757521.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for dr in tqdm_notebook(data_reference):


  0%|          | 0/61 [00:00<?, ?it/s]

## Fill in missing scholix data

In [42]:
# get names and links for scholix data mentions
working_file = './data_search_scholix/search_scholix_app_db202204_valid.csv'

data_reference, do_keys = csvh.get_csv_data(working_file)
print(do_keys)
for dr in tqdm_notebook(data_reference):
    if not 'dataset_complete' in do_keys:
        do_id = data_reference[dr]['target_id']
        data_reference[dr]["dataset_complete"]=""
        data_reference[dr]["dataset_description"]=""
        # fill in the DOI reference
        if pr_fns.valid_doi(do_id):
            print (do_id)
            data_reference[dr]['dataset_doi'] = do_id
        else:
            data_reference[dr]['dataset_doi'] = ""    
        data_reference[dr]["do_id"]=do_id
        data_reference[dr]["dataset_enddate"]=""
        if pr_fns.valid_doi(do_id):
            print (do_id)
            data_reference[dr]['dataset_location'] = "https://dx.doi.org/"+do_id
        else:
            data_reference[dr]['dataset_location'] = do_id   
        data_reference[dr]["dataset_name"]=data_reference[dr]["target_title"]
        data_reference[dr]["dataset_startdate"]=data_reference[dr]['target_published']
        data_reference[dr]["ds_type"]= ""
        if "CCDC" in data_reference[dr]["target_title"]:
            data_reference[dr]["ds_type"]= "Crystallographic Information File [cif]"
        elif "pdbe" in  do_id :
            data_reference[dr]["ds_type"]= "Crystal structure [pdbe]"
        data_reference[dr]["repository"]=""
        if "CCDC" in data_reference[dr]["target_title"]:
            data_reference[dr]["repository"]= "https://www.ccdc.cam.ac.uk"
        elif "pdbe" in  do_id :
            data_reference[dr]["repository"]= "https://www.ebi.ac.uk"
        else:
            data_reference[dr]["repository"] = do_id

# write to csv file
if len(data_reference) > 0:
    csvh.write_csv_data(data_reference, working_file)        

['pub_id', 'pub_doi', 'source_title', 'source_published', 'target_id', 'target_title', 'target_published', 'rel_type', 'duplicate', 'in_db', 'dataset_complete', 'dataset_description', 'dataset_doi', 'do_id', 'dataset_enddate', 'dataset_location', 'dataset_name', 'dataset_startdate', 'ds_type', 'repository']


C:\Users\scman1\AppData\Local\Temp\ipykernel_17256\2347414476.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for dr in tqdm_notebook(data_reference):


  0%|          | 0/27 [00:00<?, ?it/s]

## Join the references and add to DB

In [52]:
# get names and links for references in data mentions
pdf_data = Path('./data_search_pdf/pdf_mentionsapp_db202204_checked.csv')
pdf_data_refs, _ = csvh.get_csv_data(pdf_data)

collected_refs = {}
int_idx = 1
for dr in tqdm_notebook(pdf_data_refs):
    del pdf_data_refs[dr]['type']
    del pdf_data_refs[dr]['desc']
    del pdf_data_refs[dr]['data_url']
    del pdf_data_refs[dr]['action']
    del pdf_data_refs[dr]['link']
    del pdf_data_refs[dr]['issue']
    del pdf_data_refs[dr]['name']
    del pdf_data_refs[dr]['file']
    del pdf_data_refs[dr]['add']
    del pdf_data_refs[dr]['num']
    del pdf_data_refs[dr]['ID']
    del pdf_data_refs[dr]['in_db']
    collected_refs[int_idx] = pdf_data_refs[dr]
    int_idx += 1
    
# get names and links for scholix data mentions
slx_data = Path('./data_search_scholix/search_scholix_app_db202204_valid.csv')
slx_data_refs, _ = csvh.get_csv_data(slx_data)
for dr in tqdm_notebook(slx_data_refs):
    collected_refs[int_idx] = {'id':slx_data_refs[dr]['pub_id'],'doi':slx_data_refs[dr]['pub_doi'],
                               'dataset_complete':"", 'dataset_description':slx_data_refs[dr]['dataset_description'],
                               'dataset_doi':slx_data_refs[dr]['dataset_doi'], 'do_id':slx_data_refs[dr]['do_id'], 
                               'dataset_enddate':'', 'dataset_location':slx_data_refs[dr]['dataset_location'],
                               'dataset_name':slx_data_refs[dr]['dataset_name'], 
                               'dataset_startdate':slx_data_refs[dr]['dataset_startdate'], 
                               'ds_type':slx_data_refs[dr]['ds_type'], 'repository':slx_data_refs[dr]['repository']}
    int_idx += 1

out_file = Path('./data_load/data_load_202204.csv')

# write to csv file
if len(collected_refs) > 0:
    csvh.write_csv_data(collected_refs, out_file)     
    

C:\Users\scman1\AppData\Local\Temp\ipykernel_17256\1155673507.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for dr in tqdm_notebook(pdf_data_refs):


  0%|          | 0/61 [00:00<?, ?it/s]

C:\Users\scman1\AppData\Local\Temp\ipykernel_17256\1155673507.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for dr in tqdm_notebook(slx_data_refs):


  0%|          | 0/27 [00:00<?, ?it/s]

In [60]:
# get names and links for references in data mentions
in_file = Path('./data_load/data_load_202204.csv')

data_reference, _ = csvh.get_csv_data(in_file)

db_conn = dbh.DataBaseAdapter(ukchapp_db)

db_table = "datasets"
table_columns = ["dataset_complete", "dataset_description","dataset_doi","dataset_enddate", "dataset_location",
                  "dataset_name","dataset_startdate","created_at","updated_at", "ds_type", "repository"]
for dr in tqdm_notebook(data_reference):
    if data_reference[dr]['dataset_location']!= "":
        table_values = [None, data_reference[dr]['dataset_description'], data_reference[dr]['dataset_doi'], None,
                        data_reference[dr]['dataset_location'],data_reference[dr]['dataset_name'], 
                        data_reference[dr]['dataset_startdate'], "2022-05-06 18:48:00", "2022-05-06 18:48:00", 
                        data_reference[dr]['ds_type'], data_reference[dr]['repository']]
        db_conn.put_values_table(db_table, table_columns, table_values)
        #get the id of inserted record
        new_do_id = db_conn.get_value( db_table, "id", "dataset_location", data_reference[dr]['dataset_location'])[0]
        print(new_do_id)
        linktable = "article_datasets"
        linktable_columns = ["doi", "article_id", "dataset_id", "created_at", "updated_at"]
        linktable_values = [data_reference[dr]['doi'], data_reference[dr]['id'], new_do_id, "2022-05-06 18:48:00",
                            "2022-05-06 18:48:00"]
        db_conn.put_values_table(linktable, linktable_columns, linktable_values)


C:\Users\scman1\AppData\Local\Temp\ipykernel_17256\4139930805.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for dr in tqdm_notebook(data_reference):


  0%|          | 0/86 [00:00<?, ?it/s]

941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1018
1023
1024
1025
1026
